In [15]:
import nrrd
import numpy as np
import pandas as pd

In [10]:
folder_path = '../segmentations/'

In [6]:
file_name = 'Lung segmentation-10.seg.nrrd'

In [12]:
data, header = nrrd.read( folder_path + file_name )

In [41]:
right_lung = data[0]
left_lung = data[1]

In [42]:
right_lung_layer = data[0]
right_lung_mask = right_lung_layer == label_val

In [43]:
print(np.unique(right_lung_layer))      # shows all label values present
print(np.unique(right_lung_mask))       # should now be [False, True]

[0 1 2]
[False  True]


In [44]:
masks = {}
for i in range(data.shape[0]):
    seg_id = header.get(f"Segment{i}_ID")
    label_val = int(header.get(f"Segment{i}_LabelValue"))
    mask = data[i] == label_val
    masks[seg_id] = mask

In [50]:
print(masks)

{'right lung': array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, 

### Marching Cubes

In [46]:
from skimage import measure
import pyvista as pv

In [95]:
# Volume data
volume_data = masks['lungs']

# Isosurface value
iso_value = 0.05

# Extract the isosurface
vertices, faces, normals, values = measure.marching_cubes(volume_data, iso_value)

In [91]:
print(faces)

[[     2      1      0]
 [     4      3      0]
 [     0      3      2]
 ...
 [329242 329408 329245]
 [329245 329408 329409]
 [329245 329409 329238]]


In [92]:
def faces_to_edges(faces):
    edges = []
    for face in faces:
        edges.append(sorted([face[0],face[1]]))
        edges.append(sorted([face[1],face[2]]))
        edges.append(sorted([face[2],face[0]]))
    seen = set()
    unique_pairs = []
    
    for pair in edges:
        tup = tuple(pair)  # convert to tuple so it’s hashable
        if tup not in seen:
            seen.add(tup)
            unique_pairs.append(pair)
    return unique_pairs

In [96]:
edges = faces_to_edges(faces)

In [88]:
print(edges)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [97]:
# Adjust faces array for PyVista
faces_pv = np.column_stack([np.full(len(faces), 3), faces])

# Create a PyVista mesh
mesh = pv.PolyData(vertices, faces_pv)

# Visualize the mesh
mesh.plot()

Widget(value='<iframe src="http://localhost:41431/index.html?ui=P_0x76d69cff1460_6&reconnect=auto" class="pyvi…

In [59]:
edges = mesh.extract_all_edges()
lines = edges.lines.reshape((-1, 3))

In [63]:
export
for edge in lines:
    if edge[0] != 2:
        print('not 2')

In [60]:
print(lines)

[[    2     0     1]
 [    2     1     2]
 [    2     2     0]
 ...
 [    2 25858 25865]
 [    2 25861 25865]
 [    2 25865 25863]]


In [56]:
print(vertices)

[[169.05 271.   141.  ]
 [170.   271.   140.05]
 [170.   270.05 141.  ]
 ...
 [323.   287.95 114.  ]
 [322.95 311.   109.  ]
 [323.95 287.   114.  ]]
